If column K (Total Under & Undeployed) on the ‘Undeployed & Underdeployed’ tab < 0, then automatic email will alert us and show:



Coin, Column I (Total Undeployed Coins), Column J (Total Undeployed $), Column N (Under-Deployed Coins), Column O (Under-Deployed $), Column K (Total Under and Undeployed Coins), Total Under and Undeployed $ (Col K * L), Column H (Pending Withdrawals)

In [2]:
from email.mime.multipart import MIMEMultipart
import os.path
import pandas as pd
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import datetime as dt
from email.mime.text import MIMEText
import base64

In [10]:
# Connect to Drive API
spreadsheet_id = '1-2FvSx-rVma1RCZ0Hu4p7J8DJjN6TpZwz6CgNSwa7CY'
url = 'https://docs.google.com/spreadsheets/d/1-2FvSx-rVma1RCZ0Hu4p7J8DJjN6TpZwz6CgNSwa7CY'
emails = ['treasury@celsius.network',
'weekly.portfolio@celsius.network']

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None
creds = Credentials.from_authorized_user_file('token.json', SCOPES)
service = build('sheets', 'v4', credentials=creds)

# Call the Drive v3 API
SAMPLE_RANGE_NAME = 'Undeployed & Underdeployed!B2:O210'
request = service.spreadsheets().values().batchGet(
    spreadsheetId=spreadsheet_id,
    ranges=SAMPLE_RANGE_NAME,
    valueRenderOption='FORMATTED_VALUE')
response = request.execute()
df = pd.DataFrame(response['valueRanges'][0]['values']).dropna().reset_index(drop=True)

Coin, Column I (Total Undeployed Coins), Column J (Total Undeployed $), Column N (Under-Deployed Coins), Column O (Under-Deployed $), Column K (Total Under and Undeployed Coins), Total Under and Undeployed $ (Col K * L), Column H (Pending Withdrawals)

In [35]:
df2

,Coin,Total Fireblocks Balances,Main Account (minimum reserve),"Treasury (margin reserves, mining, etc)",Frictional Balances 2,User's Wallets,Pending Withdrawals,Total Undeployed (Coins),Total Undeployed ($),Total Under and Undeployed,Price,Under-Deployed (Coins),Under-Deployed ($),Total Under and Undeployed ($)
1,BTC,"13,286.34","1,310.83","4,453.41",466.71,27.33,0.26,"7,494.77","$328,232,444.17",1.450213e+04,43794.87,"7,007.36","$306,886,327.35",6.351189e+08
2,ETH,"219,631.32","12,286.06","33,910.93","3,560.13","11,148.82",172.79,"162,285.50","$511,981,553.65",2.080695e+05,3154.82,"45,784.00","$144,440,278.88",6.564218e+08
3,SNX,"6,208,773.48","109,376.04","5,466,043.30","197,605.19","10,823.76",0.00,"534,301.23","$2,960,028.82",7.040581e+05,5.54,"169,756.90","$940,453.23",3.900482e+06
4,MATIC,"17,610,266.63","7,228,397.42","60,409.72","641,217.16","500,444.16","2,202.07","9,821,015.32","$19,249,190.03",9.821015e+06,1.96,0.00,$0.00,1.924919e+07
5,DOT,"2,200,755.25","95,710.84","2,051,299.51","36,339.18","7,027.53",14.17,"46,717.36","$1,018,438.52",4.671736e+04,21.80,0.00,$0.00,1.018438e+06
6,BNT,"94,429.56","9,896.08",0.00,"86,607.62",796.64,0.00,"7,025.30","$19,530.34",1.939858e+05,2.78,"186,960.46","$519,750.09",5.392804e+05
7,CEL,"619,806,107.28",0.00,"5,016.24","7,174,806.32","1,465,436.39","1,004.04","611,160,848.32","$1,870,152,195.86",6.115267e+08,3.06,"365,822.28","$1,119,416.17",1.871272e+09
8,LUNA,"571,041.55","76,205.13",0.00,"32,388.94","1,132.52",0.00,"493,703.90","$27,469,685.27",4.937039e+05,55.64,0.00,$0.00,2.746968e+07
9,1INCH,"302,158.22","48,401.58",0.00,"91,306.67","2,408.74",0.00,"208,442.82","$396,041.35",2.084428e+05,1.90,0.00,$0.00,3.960414e+05
10,AAVE,"22,098.56","2,407.23","12,877.67","2,390.48",267.38,0.00,"6,546.28","$1,211,979.07",6.546290e+03,185.14,0.00,$0.70,1.211980e+06


In [36]:
df2.columns

Index(['Coin', 'Total Fireblocks Balances', 'Main Account (minimum reserve)',
       'Treasury (margin reserves, mining, etc)', 'Frictional Balances 2',
       'User's Wallets', 'Pending Withdrawals', 'Total Undeployed (Coins)',
       'Total Undeployed ($)', 'Total Under and Undeployed', 'Price',
       'Under-Deployed (Coins)', 'Under-Deployed ($)',
       'Total Under and Undeployed ($)'],
      dtype='object', name=0)

In [64]:
df2 = df
df2.columns = df2.loc[0]
df2 = df2.loc[1:]
cols = [col for col in df2.columns if col != '']
df2 = df2[cols]

col_float = ['Total Under and Undeployed',
             'Price']
for col in col_float:
    df2[col] = df2[col].str.replace(',','')
    df2[col] = df2[col].str.replace('$','')
    df2[col] = df2[col].astype(float)

cond = df2['Total Under and Undeployed']<0
df2['Total Under and Undeployed ($)'] = df2['Total Under and Undeployed'] * df2['Price']
cols = ['Coin',
        'Total Undeployed (Coins)',
        'Total Undeployed ($)',
        'Under-Deployed (Coins)',
        'Under-Deployed ($)',
        'Total Under and Undeployed',
        'Total Under and Undeployed ($)',
        'Pending Withdrawals']

df_error = df2[cond][cols]
df_error['Total Under and Undeployed ($)'] = df_error['Total Under and Undeployed ($)'].map(lambda x:"-${:,}".format(-x))
df_error['Total Under and Undeployed'] = df_error['Total Under and Undeployed'].map(lambda x:"{:,}".format(x))
df_error

/var/folders/q7/qhj4q23x49vb9f__hlp8t3280000gn/T/ipykernel_2124/323251918.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2[col] = df2[col].str.replace('$','')


,Coin,Total Undeployed (Coins),Total Undeployed ($),Under-Deployed (Coins),Under-Deployed ($),Total Under and Undeployed,Total Under and Undeployed ($),Pending Withdrawals
44,ZUSD,"-214,013.97","-$214,013.97",0.00,$0.00,"-214,013.97","-$214,013.97",0.00


In [65]:
# Connect to GMAIL API
client_id = "me"
SCOPES = ['https://www.googleapis.com/auth/gmail.send']
creds = None
creds = Credentials.from_authorized_user_file('token_gmail_send.json', SCOPES)

service_gmail = build('gmail', 'v1', credentials=creds)
#results = service_gmail.users().labels().list(userId=client_id).execute()
#labels = results.get('labels', [])

In [68]:
html = """
<html><body><p>Hi all,</p>
<p> Those following coins have a negative Total Under and Undeployed Value:</p>
{table}

<p> And the link to the google sheet:</p>
<a href="{url}">Deployable Metric</a>

<p>Regards,</p>
<p>CNS BOT</p>
</body></html>
"""

user_id = 'me'
subject = 'AUTOMATIC ALERT - Deployable Metric - Negative Undeployed & Underdeployed Coins'
to = 'gabriel.choukroun@celsius.network'
sender = 'gabriel.choukroun@celsius.network'


html_table = df_error.to_html(index=True,
                              header=True,
                              justify='left',
                              col_space=20)

html = html.format(table=html_table, url=url)

message = MIMEText(html, 'html')
message['to'] = to
message['from'] = sender
message['subject'] = subject
raw_message = {
    'raw':base64.urlsafe_b64encode(message.as_string().encode("utf-8")).decode("utf-8")}
 #   'payload': {'mimeType': 'text/html'}}

message = service_gmail.users().messages().send(userId=user_id, body=raw_message).execute()
print('Message Id: %s' % message['id'])

Message Id: 17ede8dd2b53396a
